<a href="https://colab.research.google.com/github/roanacla/FactorsIntegration/blob/main/Factors_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone repos

In [1]:
#Content Statistics
!git clone https://github.com/Wayne122/Content_Statistics_Demo.git
%cd /content/Content_Statistics_Demo
!pip install -r requirements.txt
!python3 -m spacy download en_core_web_lg

#Sensationalism
%cd /content/
!git clone https://github.com/roanacla/nlp_sensationalism.git
!pip install -r /content/nlp_sensationalism/requirements.txt

#Context Veracity
%cd /content/
!git clone https://github.com/snarvekark/Veracity_Factor.git
%cd /content/Veracity_Factor/
!pip install -r requirements.txt

Cloning into 'Content_Statistics_Demo'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.
/content/Content_Statistics_Demo
     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=af74a1248d72cc4530aebfe18cbb84e5320ee464b8d2ee8b5b085ec3ec2c54f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-qgup11x4/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
/content
Cloning into 'nlp_sensationalism'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 19 (delt

Note: After running the above cell, please restart the runtime. 

# Instantiation

In [1]:
import pandas as pd

#Content Statistics
%cd /content/Content_Statistics_Demo/
from Blastoff_Content_Statistics import BlastoffContentStatistics
%cd /content/
bcs = BlastoffContentStatistics()

#Sensationalims
from nlp_sensationalism.sensaScorer import SensaScorer
sensa = SensaScorer()

#Context Veracity
%cd /content/Veracity_Factor/
from context_veracity import Context_Veracity
cv = Context_Veracity()

%cd /content/

/content/Content_Statistics_Demo


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/content
Loading default pretrained model.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/content/Veracity_Factor
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/content


# Run Polinomial Equation

In [ ]:
def isFake(text):
  accur = [0.26, 0.80, 0.50]
  w = [float(i)/sum(accur) for i in accur]
  prob = []

  if text:
    in_df = pd.DataFrame(data=[text], columns=['Statement'])
    res_cs = bcs.predict(in_df).replace('pants-fire', 1.0).replace('false', 0.8).replace('barely-true', 0.6).replace('half-true', 0.4).replace('mostly-true', 0.2).replace('true', 0.0)
    prob.append(w[0] * float(res_cs.iloc[0]))
    
  if text:
    cv_val = cv.get_veracity_scores(text)
    if cv_val == 1:
      cv_final = 0.8
    elif cv_val == 2:
      cv_final = 0.4
    elif cv_val == 3:
      cv_final = 0.2
    elif cv_val == 4:
      cv_final = 1.0
    elif cv_val == 5:
      cv_final = 0.0
    elif cv_val == 0:
      cv_final = 0.6
    prob.append(w[1] * float(cv_final))
    
  if text:
    prob.append(w[2] * 0.75 * sensa.getScore(text))
    

  probTotal = sum(prob[0:len(prob)])
  return probTotal

In [ ]:
# result = isFake("Smartphones will become real personal computers")
# result = isFake("One America News Network, one of President Trump’s favorite media outlets, has been banned from posting new videos to YouTube for a week for spreading Covid-19 misinformation, YouTube said on Tuesday.")
result = isFake("Commander of the US Army: The elite of the US military will encircle China like the Soviet Union")
print("\n\n###### Result ######")
print(result)
#if result > 0.5:
#  print("FAKE NEWS ALERT!!!")
#else:
#  print("NOT FAKE")

#Result with 6 labels
if(result>0.8):
  print("pants-on-fire")
elif(result<=0.80 and result>=0.64):
  print("false")
elif(result <0.64 and result >=0.48):
  print("barely-true")
elif(result<0.48 and result>=0.32):
  print("half-true")
elif(result<0.32 and result>=(0.16)):
  print('mostly-true')
else:
  print('true')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.




###### Result ######
0.46426282051282053
half-true


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# Multi-modal model (MLP)

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=48, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

##Loading data

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1sFMftHLILNfuS0kpTlsbUzWpSNs3WkeD"})
downloaded.GetContentFile('liar_plus_dataset.zip')

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('liar_plus_dataset.zip', 'r') as myzip:
    train_data = myzip.open('train2.tsv')
    test_data = myzip.open('test2.tsv')
    valid_data = myzip.open('val2.tsv')

train_p_df = pd.read_csv(train_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')
test_p_df = pd.read_csv(test_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')
valid_p_df = pd.read_csv(valid_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')

## Rename the columns

as the module requires the data to have "Statement" and "Label" to train. If you just want to inference, then "Statement" is enough.

In [ ]:
train_p_df.rename(columns={2:'Label', 3:'Statement'}, inplace=True)
test_p_df.rename(columns={2:'Label', 3:'Statement'}, inplace=True)  

In [ ]:
X_train = train_p_df.drop('Label', axis=1)
y_train = train_p_df['Label']
X_test = test_p_df.drop('Label', axis=1)  
y_test = test_p_df['Label']               

In [ ]:
X_train = X_train.rename(columns={9: 'barelytruecounts',	10: 'falsecounts',	11: 'halftruecounts', 	12: 'mostlytruecounts', 	13: 'pantsonfirecounts'})

In [9]:
from nlp_sensationalism.sensaEncoder import SensaEncoder
sensaEncoder = SensaEncoder()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from zipfile import ZipFile


# Content Statistics
bcs_e_X_train = bcs.encode(X_train)

# Context Veracity
bcv_e_X_train = cv.liar_encode(X_train)

# Sensationalism
bse_e_X_train = sensaEncoder.getEncodedLiarLiar() #return Dataframe already
# sensaEncoder.encodeText(<text>,<dimension>)#Use this to encode a title

# Concatenate
bcs_e_X_train = pd.DataFrame(data=bcs_e_X_train)
bcv_e_X_train = pd.DataFrame(data=bcv_e_X_train)

e_X_train = pd.concat([bcs_e_X_train, bcv_e_X_train, bse_e_X_train], axis=1) ## Also add it here

# label
d_y_train = pd.get_dummies(y_train)
e_X_test = bcs.encode(X_test)      
d_y_test = pd.get_dummies(y_test)  

In [ ]:
e_X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
0,7.211837,4.940129,6.629362,9.050505,7.779743,5.332962,7.848826,2.241910,4.794161,4.960108,7.893267,10.353553,3.282789,6.091529,6.585247,7.080389,0.0,3.090829,0.000000,0.522613,0.000000,0.000000,0.261555,1.204743,0.000000,0.0,0.000000,0.000000,0.0,1.557130,0.000000,0.858425,-0.145043,-0.360949,-0.117985,0.161849,-0.182158,0.692266,0.390122,0.010706,0.128772,0.126779,-0.362850,-0.146944,-0.208223,-0.495680,-0.603125,0.482074
1,8.685092,24.993542,8.810983,15.461455,18.635612,17.006910,12.319558,12.790240,13.600662,16.159721,24.423901,0.000000,4.217362,10.460907,7.305364,15.332805,0.0,0.000000,0.551194,0.901265,1.205963,1.269742,1.040393,0.000000,0.568101,0.0,0.363856,2.054306,0.0,0.000000,0.253727,0.056773,-0.654220,-0.135472,0.240176,0.284358,0.301712,0.030432,0.982796,1.079335,-0.491665,0.192161,-0.459778,-0.532213,0.090021,0.661293,-0.517795,-0.221878
2,10.717841,17.493813,21.625214,19.902271,16.396116,18.681686,9.622036,15.566667,16.872272,2.891501,19.356615,5.649988,12.538577,12.147779,13.704709,14.683738,0.0,123.392120,63.841248,99.410301,41.583771,54.305538,70.684692,57.173073,0.000000,0.0,0.000000,37.758835,0.0,35.323105,0.000000,81.449501,-1.036402,-0.499439,-0.495896,0.526013,0.085208,0.504849,0.274258,0.585883,0.109990,0.015748,-0.297879,-1.042105,0.454494,-0.408630,-1.007253,-0.327223
3,7.525606,7.729619,12.469887,8.893851,4.215873,9.874451,8.484407,7.388212,3.339989,5.688490,9.019494,9.228015,0.000000,5.112737,0.272345,11.246958,0.0,5.650431,0.686527,2.626606,0.403262,0.888433,1.759920,2.395559,0.000000,0.0,0.000000,0.406031,0.0,2.275555,0.000000,2.573129,0.259972,0.126702,-0.223310,-0.417592,0.119613,0.818704,0.665083,0.125510,-0.526279,-0.132033,-0.567343,0.076380,0.186313,-0.024057,0.067190,0.451633
4,8.746030,6.962137,8.937541,10.381464,12.112238,9.934958,11.205392,5.096816,2.349538,8.490510,8.046690,6.587426,6.181075,9.011739,6.192536,5.939188,0.0,123.392120,63.841248,99.410301,41.583771,54.305538,70.684692,57.173073,0.000000,0.0,0.000000,37.758835,0.0,35.323105,0.000000,81.449501,-1.087654,-0.179319,0.102155,-0.504031,-0.043728,0.060955,-0.133286,0.089309,-0.069125,-0.879518,-0.286424,0.202886,0.067224,0.301671,0.068741,0.335711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,14.320944,9.797541,13.178794,9.223291,12.944340,8.568989,13.726956,5.387570,6.286666,6.619838,11.021467,10.266586,11.548632,5.286120,8.096165,9.989305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.152220,-0.803319,-0.895731,0.608661,0.745475,0.378171,0.808447,0.997398,-0.370447,0.598768,-0.119378,-0.415833,-0.221954,0.182336,-0.350657,-0.402497
10236,7.283374,2.749018,13.989727,9.975429,7.808156,13.104544,15.540765,10.433757,7.728669,12.493134,12.843208,10.414503,15.673471,3.529014,12.453532,8.305533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.538382,-0.594273,0.352540,-0.120017,-0.149096,0.541178,0.812760,0.923694,-0.646873,-0.045234,-0.590301,-0.646457,-0.485697,-0.180688,-0.499646,0.242648
10237,14.668739,17.153061,18.849628,16.924801,16.816706,18.075962,23.956923,9.297574,15.733624,23.805605,15.216730,13.481298,15.414532,13.886168,24.709255,33.960194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.732812,-1.283967,0.197211,0.274080,0.434829,-0.049085,1.052436,0.503341,-0.963324,0.844328,-0.246956,-0.600262,0.502401,-0.496158,-0.838488,-0.496424
10238,8.766872,4.232332,16.911827,12.412842,10.520018,15.297036,18.743952,16.318684,6.590806,11.281427,15.499956,10.263979,20.670725,6.094724,11.486252,19.297413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.188583,0.436562,-0.090641,-0.102353,0.326288,-0.304644,0.643151,0.565106,-0.146242,0.376583,-0.292953,-0.068282,0.389697,1.096096,-0.521289,-0.0784

In [ ]:
d_y_train

,barely-true,false,half-true,mostly-true,pants-fire,true
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
10237,0,0,0,1,0,0
10238,0,0,0,1,0,0
10239,0,0,1,0,0,0
10240,0,1,0,0,0,0


## Fit

In [ ]:
model.fit(e_X_train, d_y_train,
          epochs=1000,
          batch_size=8,
          shuffle=True)

Epoch 1/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7714
Epoch 2/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7580
Epoch 3/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7581
Epoch 4/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7579
Epoch 5/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7573
Epoch 6/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7572
Epoch 7/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7573
Epoch 8/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7572
Epoch 9/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7570
Epoch 10/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7570
Epoch 11/1000
1280/1280 [==============================] - 2s 2ms/step - loss: 1.7571
Epoch 12/1000
1280/1280 [==============================] - 2s 2

## Save & Load Model

In [ ]:
#SAVE KERAS MODEL
from tensorflow import keras
model.save('/content/MODEL')

INFO:tensorflow:Assets written to: /content/MODEL/assets


In [ ]:
#ZIP MODEL
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))


zipf = zipfile.ZipFile('model.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('./MODEL/', zipf)
zipf.close()

In [3]:
#DOWNLOAD MODEL FROM GOOGLE SHARED DRIVE
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1tPWFaGKF7Ibiq-kP897Uh7Nk0Yix6jFX',
                                  dest_path='./mlp_model.zip',
                                  unzip=False)

In [4]:
#UNZIP
import zipfile
with zipfile.ZipFile('/content/mlp_model.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [5]:
#LOAD MODEL
from tensorflow import keras
mlp_model = keras.models.load_model('/content/MODEL/')

## Inference

In [11]:
import numpy as np
from IPython.display import Image, clear_output  # to display images
from nlp_sensationalism.sensaEncoder import SensaEncoder

def MLPInference(title, model):
    bcs = BlastoffContentStatistics()
    sensaEncoder = SensaEncoder()
    cv = Context_Veracity()
    labels = ['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'true']

    # Sensationalism
    
    a = sensaEncoder.encodeText(title, dimension=16).numpy()
    sdf = pd.DataFrame()
    sdf = sdf.append({'a':a[0],'b':a[1],'c':a[2],'d':a[3],'e':a[4],
                      'f':a[5],'g':a[6],'h':a[7],'i':a[8],'j':a[9],
                      'k':a[10],'l':a[11],'m':a[12],'n':a[13],'o':a[14],
                      'p':a[15]}, ignore_index=True)
    bse_e_X_train = sdf

    X_train = pd.DataFrame(data=[title], columns=['Statement'])
    # print(X_train)
    # Content Statistics
    bcs_e_X_train = bcs.encode(X_train)

    # # Context Veracity
    bcv_e_X_train = cv.encode(X_train)
    
    # # Concatenate
    bcs_e_X_train = pd.DataFrame(data=bcs_e_X_train)
    bcv_e_X_train = pd.DataFrame(data=bcv_e_X_train)
    # bse_e_X_train = pd.DataFrame(data=bse_e_X_train)

    e_X_train = pd.concat([bcs_e_X_train, bcv_e_X_train, bse_e_X_train], axis=1)

    pred = model.predict(e_X_train)
    clear_output()

    return labels[np.argmax(pred[0])]

In [12]:
MLPInference('Media fawns over Biden\'s Cabinet rollout, describes being rescued from this craziness by superheroes', model=mlp_model)


'half-true'

# How did we train our models?

Han-Wei Lin [Content Statistics] -> https://colab.research.google.com/drive/1pOhnmm6X_J4t__2VxSISoDGTtWa_gZ5-?usp=sharing

Roger Navarro [Sensationalism] -> https://github.com/roanacla/ML_Sensationalism/blob/main/Sensationalism.ipynb

Sheetal Narvekar [Context Veracity] -> https://github.com/snarvekark/Veracity_Factor/blob/master/Context_Veracity_Main.ipynb